# Experimentation with web scraping Indeed

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
url = "https://www.indeed.com/jobs?q=data+scientist+intern&l=New+York&radius=50&sort=date"


In [11]:
page = requests.get(url)

In [12]:
soup = BeautifulSoup(page.text, "html.parser")

In [217]:
"Previous" in soup.find(name="span", attrs={"class": "np"}).text

False

In [75]:
n_jobs = c.split(" ")[-2]

In [79]:
int((n_jobs).replace(",", ""))

5379

In [13]:
for div in soup.find_all(name="div", attrs={"class":"row"}):
    #Date
    date_tag = div.find(name="span", attrs={"class": "date"})
    print(date_tag.text)


Just posted
1 day ago
4 days ago
12 days ago
9 days ago
10 days ago
9 days ago
11 days ago
10 days ago
13 days ago


In [238]:
d = soup.find(name="span", attrs={"class":"np"})
d

<span class="np">Next »</span>

In [240]:
d = soup.find(name="div", attrs={"class": "pagination"})
d.find_all("a")[-1]["href"]

'/jobs?q=data+scientist+intern&l=austin&radius=50&sort=date&start=10'

In [239]:
d.find_all("a")
#len(d.find_all("a"))
#d= d.find_all("a")[-1]["href"]
#e = d.find(name="span", attrs={"class":"np"})
print(d)

<span class="np">Next »</span>


In [5]:
import pandas as pd

In [77]:
df = pd.DataFrame(columns = ["job_title", "location", "company", "url", "jobsite", "job_description"])

In [78]:
df.to_csv("data/listings_data.csv",index=False)

In [15]:
df2 = pd.read_csv("data/listings_data.csv")

In [16]:
df2

,job_title,location,company,url,jobsite


In [165]:
x = None

In [168]:
if not x:
    print("Helo")

Helo


In [203]:
dtf = df
for div in soup.find_all(name="div", attrs={"class": "row"}):
    listing = get_listing_info(div)
    other = pd.DataFrame(listing)
    dtf = dtf.append(other)
dtf

,company,job_title,location,url
0,USAA,Business Intelligence Analyst,San Antonio TX 78288,/pagead/clk?mo=r&ad=-6NYlbfkN0CAlkOQ68lM1c9i6U...
0,Adaptivestack Technologies Inc.,Electronic Data Interchange (EDI) Business Ana...,Austin TX,/company/Adaptivestack-Technologies-Inc./jobs/...
0,Simple [A],QA Engineer / QA Analyst (remote),Austin TX 78701 (Downtown area),/rc/clk?jk=62e89d4367865679&fccid=d271b8ad811e...
0,Advanced Micro Devices Inc.,Sr. Business Analyst,Austin TX,/rc/clk?jk=0ec31aa7e9a539b7&fccid=b45c4b5a9c9a...
0,Google,Sales Engineering Manager Shared Services Goog...,Austin TX,/rc/clk?jk=fa6e92598f23922e&fccid=a5b4499d9e91...
0,Central Waste Management,On-Call Trash Service Valet - TRUCK REQUIRED -...,Austin TX 78750,/company/Central-Waste-Management/jobs/Call-Tr...
0,Spectrum,Mgr Network Engineering,Austin TX,/rc/clk?jk=61a2811db9e0bb9e&fccid=c2d1b0769a53...
0,IBM,Data Security Consultant,Austin TX 73344,/rc/clk?jk=04c2935876648d8d&fccid=de71a49b535e...
0,IBM,Research Staff Member Artificial Intelligence,Austin TX 73344,/rc/clk?jk=1d82d66876b6d203&fccid=de71a49b535e...
0,IBM,Associate Partner Advanced Analytics/Data Science,Austin TX 73344,/rc/clk?jk=61c9e8394c055ecc&fccid=de71a49b535e...


In [202]:
def get_listing_info(div):
    """
    Get the results of scraping a single listings job listing.
    :param div: the contents of a single listing's div tag
    :return: dict, the desired scraping information
    """
    return {"job_title": [get_job_title(div)],
            "location": [get_location(div)],
            "company": [get_company_name(div)],
            "url": [get_url_link(div)]}


def get_number_of_jobs(soup):
    """
    Get the number of jobs returned for the search criteria
    :param soup: soup object for the first search page
    :return: int, the number of jobs that the search returns
    """
    tag = soup.find("div", {"id": "searchCount"})
    n_jobs = tag.text.split(" ")[-2]
    return int(n_jobs.replace(",", ""))


def get_job_title(div):
    """
    Extract the job titles from the div tag.
    :param div: tag object, the div tag from a job posting
    :return: string, the job title
    """
    a = div.find(name="a", attrs={"data-tn-element": "jobTitle"})
    return a["title"].replace(",", "")


def get_company_name(div):
    """
    Extract company names from the div tag.
    :param div: tag object, the div tag from a job posting
    :return: string, the company name
    """
    company = div.find(name="span", attrs={"class": "company"})
    a = div.find(name="a", attrs={"data-tn-element": "companyName"})
    if not a:
        return ' '.join(company.text.split()).replace(",", "")
    else:
        return ' '.join(a.text.split()).replace(",", "")


def get_url_link(div):
    """
    Extract the URL link from the div tag.
    :param div: tag object, the div tag from a job posting
    :return: string, the URL of the job posting
    """
    a = div.find(name="a", attrs={"data-tn-element": "jobTitle"})
    return a["href"]


def get_location(div):
    """
    Extract the location of the job posting from the div tag.
    :param div: tag object, the div tag from a job posting
    :return: string, the location of the job posting
    """
    location = div.find(name="span", attrs={"class": "location"})
    return location.text.replace(",", "")

In [79]:
df = pd.read_csv("data/listings_data.csv")

In [80]:
df

,company,job_description,job_title,jobsite,location,url
0,IBM,Job Description\nIBM LOGO\nDO YOUR BEST WORK E...,2018 Data Scientist Internship,Indeed,Austin TX 73344,/rc/clk?jk=7efe33d6388ee124&fccid=de71a49b535e...
1,IBM,Job Description\nIBM LOGO\nDO YOUR BEST WORK E...,2018 Senior Data Scientist Internship,Indeed,Austin TX 73344,/rc/clk?jk=c9d2d46405354ccd&fccid=de71a49b535e...
2,IBM,Job Description\nIBM LOGO\nDO YOUR BEST WORK E...,Cognitive / AI / Machine Learning Software Dev...,Indeed,Austin TX 73344,/rc/clk?jk=f703d866bcbe77f9&fccid=de71a49b535e...
3,IBM,Job Description\nIBM LOGO\nDO YOUR BEST WORK E...,2018 Sr. Cognitive /AI/ Machine Learning Softw...,Indeed,Austin TX 73344,/rc/clk?jk=b498447940edee49&fccid=de71a49b535e...
4,IBM,Job Description\nIBM LOGO\nDO YOUR BEST WORK E...,Watson Health - Data Scientist,Indeed,Austin TX 73344,/rc/clk?jk=c1bc88d1fbfd8c88&fccid=de71a49b535e...
5,Texan by Nature,Data Analysis Intern job - Texan by Nature - A...,Data Analysis Intern,Indeed,Austin TX,/company/Texan-by-Nature/jobs/Data-Analysis-In...
6,CDK Global,Software Development Internship - Austin\nEngl...,Software Development Internship - Austin,Indeed,Austin TX,/rc/clk?jk=1cebb889b301d804&fccid=cf4c744760f4...
7,Colorado River Alliance,Program Coordinator #0007 job at Colorado Rive...,Program Coordinator,Indeed,Austin TX,/rc/clk?jk=c043d42b087af77e&fccid=1153f9b6a570...
8,Charles Schwab,2018 Schwab Intern Academy: Analytics and Busi...,2018 Schwab Intern Academy: Analytics and Busi...,Indeed,Austin TX 73344,/rc/clk?jk=16079903e7c0f8c6&fccid=3c74eafe288f...
9,Nano Global,Nano Global - Jobs: Engineering Product Develo...,Engineering Product Development Intern,Indeed,Austin TX,/rc/clk?jk=eee8a480cce817fd&fccid=ee3c12953393...


In [83]:
df["job_description"].iloc[12]

"Software Developer Internship\nin\nAustin, TX\n| CACI International\nSmall Business\xa0\xa0|\nSuppliers\nAbout Us\n|\nWhat We Do\n|\nMedia & Events\n|\nInvestors\n|\nCareers\n|\nContracts\n|\nContact Us\nJoin Our Talent Network\nAbout Us\nWhat We Do\nMedia & Events\nInvestors\nCareers\nContracts\nContact Us\nSmall Business\nSuppliers\nCareers\n| Software Developer Internship - Austin, TX\nShare This Job\nApply Now\nSoftware Developer Internship\nReq #: 200563\nLocation:\nAustin,\nTX\nUS\nJob Category: Administration\nSecurity Clearance: None\nClearance Status: Not Applicable\nJob DescriptionSoftware Developer Internship\xa0Internship Opportunity: Are you a problem solver, ready for a challenge this summer? Ticom Geomatics (TGI) is seeking 2018 Summer Interns for Software Developer Internships! TGI is an Austin-based company and recognized industry leader providing Mission-Ready Geolocation and ISR Systems and Services to Government and Industry. Through our Intern program's on the job

In [43]:
def url_req(i):
    url2 = df["url"].iloc[i]
    page = requests.get(''.join(["https://www.indeed.com", url2]))
    return BeautifulSoup(page.text, "html.parser")

In [59]:
url = df["url"].iloc[0]
url

'/rc/clk?jk=7efe33d6388ee124&fccid=de71a49b535e21cb'

In [60]:
page = requests.get(''.join(["https://www.indeed.com", url]))

In [61]:
soup = BeautifulSoup(page.text, "html.parser")

In [62]:
soup.text

'\n\n\n\n\r\n        body\r\n        {\r\n            font-family: Arial, Verdana;\r\n            font-size: 12px;\r\n        }\r\n        .title\r\n        {\r\n            font-style: italic;\r\n            font-size: 16px;\r\n        }\r\n        .error\r\n        {\r\n            padding-left: 5px;\r\n            color: #FF0000;\r\n        }\r\n    \n\r\n        function tn_ShowPopupMessage(sender, msg) {\r\n            var r = $find(\'rttMsg\');\r\n            r.set_content(\'<div class="tnReferralMsg">\' + msg + \'</div>\');\r\n            setTimeout(function () { r.show(); }, 100);\r\n        }\r\n\r\n        function DisableButtons() {\r\n            var inputs = document.getElementsByTagName("INPUT");\r\n            for (var i in inputs) {\r\n                if (inputs[i].type === "button" || inputs[i].type === "submit") {\r\n                    inputs[i].disabled = true;\r\n                }\r\n            }\r\n        }\r\n\r\n        window.onbeforeunload = DisableButtons;\

In [54]:
soup = url_req(10)

In [58]:
soup.text

'\n\n\nRocket Software Careers\n\n\n\n\n\nfunction clear_search() {document.search_form.search_phrase.value= "";}\n\r\nwindow.onload=function()\r\n{\r\n\r\nif(""=="yes")\r\n{\r\n  \r\n}\r\n}\r\n\n\n\n\n\n\ntry{Typekit.load();}catch(e){}\n\n\n\n\n\n\n\n\n\n\nvar productVersion = "2017.7"; var timeoutSeconds = 3600; var timeoutWarningSeconds = 300; var displayTimeWarning = 0; var isCandidateLoggedIn = false; var companyId = 16509; var ep_version = 1;\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\nHomeCareersAbout Rocket SoftwareHelpful Hints for Applicants\n\n\n\n \n\n\nSearch Current OpeningsUpdate Your ProfileCreate a Job AgentUpdate Job Agent\n\xa0 \r\n\n\n\nYour user session will timeout in 5 minutes.  Please click OK to continue with your application.\n\n\r\n/* <![CDATA[ */\r\nfunction URLReplacePlus(strIn) {\r\n\tvar a = "+";\r\n\tvar b = "%43";\r\n\tvar c = strIn;\r\n\tvar i = c.indexOf(a);\r\n\tvar l = b.length;\r\n\twhile (i != -1) { c = c.substring(0,i) + b + c.substring(i + a.length,c.

In [74]:
import urllib

html = requests.get(''.join(["https://www.indeed.com", url]))
soup = BeautifulSoup(html.text, "html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)
print(text)

Job Description
IBM LOGO
DO YOUR BEST WORK EVER.
Job DescriptionSTART AND END DATES FOR THIS INTERNSHIP ARE SPRING 2018 (6 Months), SUMMER 2018 (3 Months) and FALL 2018 (6 Months)At IBM we have an amazing opportunity to transform the world with cognitive technology.
By using the vast amounts of information available today to identify new patterns and make new discoveries, we are helping cities become smarter, hospitals transform patient care, financial institutions minimize risk, and pharmaceuticals find cures for rare diseases.
Join the forward-thinking teams at IBM solving some of the world’s most complex problems – there is no better place to grow your career!Data Scientist Interns extract knowledge or insights from structured or unstructured data.
They draw upon the practice of data analysis, using predictive analytics, data mining, pattern recognition, data modeling, machine learning and various statistical methods in order to solve large scale optimization problems and to underst